In [3]:
import jittor as jt
from jittor.optim import Optimizer
from jittor import nn
from jittor import Module
import random
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from jittor.dataset.cifar import CIFAR10
from jittor.dataset import DataLoader
import jittor.transform as trans


[i 0407 21:26:19.027983 20 log.cc:351] Load log_sync: 1
[i 0407 21:26:19.033725 20 compiler.py:956] Jittor(1.3.8.5) src: /root/miniconda3/lib/python3.10/site-packages/jittor
[i 0407 21:26:19.037624 20 compiler.py:957] g++ at /usr/bin/g++(11.4.0)
[i 0407 21:26:19.038409 20 compiler.py:958] cache_path: /root/.cache/jittor/jt1.3.8/g++11.4.0/py3.10.8/Linux-5.15.0-9x14/IntelRXeonRGolx95/default
[i 0407 21:26:19.042383 20 __init__.py:411] Found nvcc(12.1.105) at /usr/local/cuda/bin/nvcc.
[i 0407 21:26:19.045575 20 __init__.py:411] Found addr2line(2.38) at /usr/bin/addr2line.
[i 0407 21:26:19.192012 20 compiler.py:1011] cuda key:cu12.1.105_sm_89
[i 0407 21:26:19.709901 20 __init__.py:227] Total mem: 1007.52GB, using 16 procs for compiling.
[i 0407 21:26:19.890676 20 jit_compiler.cc:28] Load cc_path: /usr/bin/g++
[i 0407 21:26:20.073458 20 init.cc:62] Found cuda archs: [89,]


In [4]:
jt.flags.use_cuda=1

[i 0407 21:26:25.527148 20 cuda_flags.cc:49] CUDA enabled.


In [5]:
class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(
            in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3,
                               stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion*planes)
            )

    def execute(self, x):
        out = nn.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = nn.relu(out)
        return out


class Bottleneck(nn.Module):
    expansion = 4

    def __init__(self, in_planes, planes, stride=1):
        super(Bottleneck, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3,
                               stride=stride, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)
        self.conv3 = nn.Conv2d(planes, self.expansion *
                               planes, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(self.expansion*planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion*planes)
            )

    def execute(self, x):
        out = nn.relu(self.bn1(self.conv1(x)))
        out = nn.relu(self.bn2(self.conv2(out)))
        out = self.bn3(self.conv3(out))
        out += self.shortcut(x)
        out = nn.relu(out)
        return out


class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=10):
        super(ResNet, self).__init__()
        self.in_planes = 64

        self.conv1 = nn.Conv2d(3, 64, kernel_size=3,
                               stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)
        self.linear = nn.Linear(512*block.expansion, num_classes)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes * block.expansion
        return nn.Sequential(*layers)

    def execute(self, x):
        out = nn.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = nn.avg_pool2d(out, 4)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out


def ResNet18():
    return ResNet(BasicBlock, [2, 2, 2, 2])


def ResNet34():
    return ResNet(BasicBlock, [3, 4, 6, 3])


def ResNet50():
    return ResNet(Bottleneck, [3, 4, 6, 3])


def ResNet101():
    return ResNet(Bottleneck, [3, 4, 23, 3])


def ResNet152():
    return ResNet(Bottleneck, [3, 8, 36, 3])

In [6]:
def train(net,train_data_loader,optimizer,total_train_step,epoch,compose):
    net.train()
    for data in train_data_loader:
        imgs,targets=data
        imgs,targets=imgs.float32(),targets.float32()
        imgs=imgs.permute(0,3,1,2)
        imgs=compose(imgs)
        outputs=net(imgs)
        loss=nn.cross_entropy_loss(outputs,targets)
        optimizer.step(loss)
        total_train_step+=1
        if total_train_step %160 ==0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                    epoch, total_train_step*32, len(train_data_loader),
                    100. * total_train_step*32 / len(train_data_loader), loss.data[0]))
            #format_text='Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}\n'.format(
                    #epoch, total_train_step*32, len(train_data_loader),
                    #100. * total_train_step*32 / len(train_data_loader), loss.data[0])
            #file.write(format_text)

def test(net,test_data_loader,epoch,compose):
    net.eval()
    total_acc = 0
    total_num = 0
    for batch_idx, (inputs, targets) in enumerate(test_data_loader):
        batch_size = inputs.shape[0]
        inputs,targets=inputs.permute(0,3,1,2).float32(),targets.float32()
        inputs=compose(inputs)
        outputs = net(inputs)
        pred = np.argmax(outputs.data, axis=1)
        acc = np.sum(targets.data==pred)
        total_acc += acc
        total_num += batch_size
        acc = acc / batch_size
        if batch_idx % 64 == 0:
            print('Test Epoch: {} [{}/{} ({:.0f}%)]\tAcc: {:.6f}'.format(epoch, \
                    batch_idx*batch_size, len(test_data_loader),100. * float(batch_idx)*batch_size / len(test_data_loader), acc))
    print ('Total test acc =', total_acc / total_num)
    #format_text=f'Total test acc ={total_acc / total_num}\n'
    #file.write(format_text)
    return total_acc/total_num

In [7]:
net=ResNet18()
train_data=CIFAR10(train=True)
test_data=CIFAR10(train=False)
compose=trans.Compose([trans.ImageNormalize((0.485,0.456,0.406),(0.229, 0.224, 0.225))])
train_data_loader=DataLoader(train_data,batch_size=32)
test_data_loader=DataLoader(test_data,batch_size=32)
learning_rate=1e-3
optimizer=nn.SGD(net.parameters(),lr=learning_rate,momentum=0.9,weight_decay=5e-4)
scheduler = jt.lr_scheduler.StepLR(optimizer, step_size=30, gamma=0.1)
total_train_step=0
total_test_step=0
test_acc=[]
epochs=100
for epoch in range(epochs):
    print("epochs:{}".format(epoch+1))
    train(net,train_data_loader,optimizer,total_train_step,epoch+1,compose)
    test_acc.append(test(net, test_data_loader, epoch+1,compose))
    scheduler.step()

Files already downloaded and verified
Files already downloaded and verified
epochs:1
Train Epoch: 1 [5120/50000 (10%)]	Loss: 1.869271
Train Epoch: 1 [10240/50000 (20%)]	Loss: 1.865712
Train Epoch: 1 [15360/50000 (31%)]	Loss: 1.208960
Train Epoch: 1 [20480/50000 (41%)]	Loss: 1.450563
Train Epoch: 1 [25600/50000 (51%)]	Loss: 0.887372
Train Epoch: 1 [30720/50000 (61%)]	Loss: 1.283290
Train Epoch: 1 [35840/50000 (72%)]	Loss: 1.073344
Train Epoch: 1 [40960/50000 (82%)]	Loss: 0.967505
Train Epoch: 1 [46080/50000 (92%)]	Loss: 0.918199
Test Epoch: 1 [0/10000 (0%)]	Acc: 0.750000
Test Epoch: 1 [2048/10000 (20%)]	Acc: 0.531250
Test Epoch: 1 [4096/10000 (41%)]	Acc: 0.531250
Test Epoch: 1 [6144/10000 (61%)]	Acc: 0.687500
Test Epoch: 1 [8192/10000 (82%)]	Acc: 0.687500
Total test acc = 0.6627
epochs:2
Train Epoch: 2 [5120/50000 (10%)]	Loss: 1.333475
Train Epoch: 2 [10240/50000 (20%)]	Loss: 0.823460
Train Epoch: 2 [15360/50000 (31%)]	Loss: 0.714010
Train Epoch: 2 [20480/50000 (41%)]	Loss: 1.014778
Tra

In [ ]:
plt.plot(test_acc,'r',label="test_acc")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend()
plt.show()